In [20]:
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from nltk import ngrams
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Eric\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def tokenizeFile(fileName: str) -> list:
    '''Will take in the name of a txt file located in the base directory 
    of the drive and return a list of tokens. Tokens are defined to be
    any nonzero sequence of characters.'''
    tokens = []
    with open(fileName, 'r', encoding="ISO-8859-1") as file:
        tokens = word_tokenize(file.read())
    return list(tokens)

In [34]:
# Cut number of words so we can fit everything in memory
tokens_all = tokenizeFile("TheLordOfTheRings_Book1.txt")
tokens = tokens_all[:len(tokens_all)//50]
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

# Change this parameter to decrease or increase the size of the training samples
N = 30
ngrams_tokens = list(ngrams(tokens, N))
np.random.shuffle(ngrams_tokens)
print('Total ngrams: %d' % len(ngrams_tokens))

# Combine tokens into training sample phrases
ngrams_samples = [" ".join(x) for x in ngrams_tokens]
print("Total Samples: %d" % len(ngrams_samples))

Total Tokens: 4435
Unique Tokens: 1261
Total ngrams: 4406
Total Samples: 4406


Assign a unique ID to each token and make an inverted index so we can look up terms easily later.

Determine the vocabulary size for the Embedding layer.

In [35]:
tokenizer = Tokenizer(filters='\t\n')
tokenizer.fit_on_texts(ngrams_samples)
sequencies = np.array(tokenizer.texts_to_sequences(ngrams_samples))
# sequencies = tokenizer.texts_to_sequences(ngrams_samples)
vocab_size = len(tokenizer.word_index) + 1
print("Sequences Shape: " + str(sequencies.shape))
print("Vocabulary size: %d" % vocab_size)

Sequences Shape: (4406, 30)
Vocabulary size: 1190


Split into training, validation, and testing

Split into input and expected output where the input is all but the last token and the output is the last token

In [36]:
from numpy import array as ar

tr = int(len(sequencies) * 0.9) # Training samples is 90% of data
va = int(len(sequencies) * 0.95) # Validation samples is 5% of data 
te = len(sequencies) # Testing samples is last 5% of data
training, validation, testing = sequencies[:tr], sequencies[tr:va], sequencies[va:te]

tr_input, tr_output = ar([x[:-1] for x in training]), [x[-1] for x in training]
tr_output = to_categorical(tr_output, num_classes=vocab_size)

va_input, va_output = ar([x[:-1] for x in validation]), [x[-1] for x in validation]
va_output = to_categorical(va_output, num_classes=vocab_size)

te_input, te_output = ar([x[:-1] for x in testing]), [x[-1] for x in testing]
te_output = to_categorical(te_output, num_classes=vocab_size)

In [37]:
seq_length = tr_input.shape[1]
print(seq_length)

29


In [38]:
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=seq_length))
model.add(LSTM(300, return_sequences=True))
model.add(LSTM(300))
model.add(Dense(300, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 29, 300)           357000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 29, 300)           721200    
_________________________________________________________________
lstm_8 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_7 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_8 (Dense)              (None, 1190)              358190    
Total params: 2,247,890
Trainable params: 2,247,890
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(tr_input, tr_output, batch_size=32, epochs=100, validation_data=(va_input, va_output))

Train on 3965 samples, validate on 220 samples
Epoch 1/100
3965/3965 [==============================] - 12s 3ms/step - loss: 6.1290 - acc: 0.0633 - val_loss: 5.8044 - val_acc: 0.0591
Epoch 2/100
3965/3965 [==============================] - 10s 3ms/step - loss: 5.6057 - acc: 0.0678 - val_loss: 5.9320 - val_acc: 0.0591
Epoch 3/100
3965/3965 [==============================] - 10s 3ms/step - loss: 5.4021 - acc: 0.0890 - val_loss: 5.7602 - val_acc: 0.0773
Epoch 4/100
3965/3965 [==============================] - 10s 3ms/step - loss: 5.1864 - acc: 0.1034 - val_loss: 5.8424 - val_acc: 0.0909
Epoch 5/100
3965/3965 [==============================] - 10s 3ms/step - loss: 5.0544 - acc: 0.1100 - val_loss: 5.9152 - val_acc: 0.0909
Epoch 6/100
3965/3965 [==============================] - 10s 3ms/step - loss: 4.9360 - acc: 0.1160 - val_loss: 6.0030 - val_acc: 0.1091
Epoch 7/100
3965/3965 [==============================] - 10s 3ms/step - loss: 4.7995 - acc: 0.1294 - val_loss: 6.1126 - val_acc: 0.1091
E

3965/3965 [==============================] - 10s 3ms/step - loss: 0.0131 - acc: 1.0000 - val_loss: 12.1827 - val_acc: 0.0773
Epoch 61/100
3965/3965 [==============================] - 10s 3ms/step - loss: 0.0116 - acc: 1.0000 - val_loss: 12.1996 - val_acc: 0.0727
Epoch 62/100
3965/3965 [==============================] - 10s 3ms/step - loss: 0.0105 - acc: 1.0000 - val_loss: 12.2246 - val_acc: 0.0727
Epoch 63/100
3965/3965 [==============================] - 10s 3ms/step - loss: 0.0094 - acc: 1.0000 - val_loss: 12.2739 - val_acc: 0.0773
Epoch 64/100
3965/3965 [==============================] - 10s 3ms/step - loss: 0.0085 - acc: 1.0000 - val_loss: 12.2823 - val_acc: 0.0727
Epoch 65/100
3965/3965 [==============================] - 10s 3ms/step - loss: 0.0077 - acc: 1.0000 - val_loss: 12.3341 - val_acc: 0.0773
Epoch 66/100
3965/3965 [==============================] - 10s 2ms/step - loss: 0.0069 - acc: 1.0000 - val_loss: 12.3552 - val_acc: 0.0727
Epoch 67/100
3965/3965 [=======================

In [40]:
from random import randint
seed_text = ngrams_samples[randint(0, len(ngrams_samples))]
print(seed_text)

of Dearth ( 1158-60 ) were at the time of this tale long past and the Hobbits had again become accustomed to plenty . The land was rich and kindly


In [41]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

, and though it had to been deserted when they entered it , it had before been well in order to provide the necessary background of 'history to its manner , and always plodded on , mostly by night , till i stood by balin 's tomb in moria .
